In [55]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import random
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [43]:
url = "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"
df = pd.read_csv(url)
df.head(20)

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55
5,2018-09-21,235.00,237.00,227.95,233.75,234.60,5395319,12589.59
6,2018-09-19,235.95,237.20,233.45,234.60,234.90,1362058,3202.78
7,2018-09-18,237.90,239.25,233.50,235.50,235.05,2614794,6163.70
8,2018-09-17,233.15,238.00,230.25,236.40,236.60,3170894,7445.41
9,2018-09-14,223.45,236.70,223.30,234.00,233.95,6377909,14784.50


In [44]:
df.describe()

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
count,2035.000000,2035.000000,2035.000000,2035.000000,2035.00000,2.035000e+03,2035.000000
mean,149.713735,151.992826,147.293931,149.474251,149.45027,2.335681e+06,3899.980565
std,48.664509,49.413109,47.931958,48.732570,48.71204,2.091778e+06,4570.767877
min,81.100000,82.800000,80.000000,81.000000,80.95000,3.961000e+04,37.040000
25%,120.025000,122.100000,118.300000,120.075000,120.05000,1.146444e+06,1427.460000
50%,141.500000,143.400000,139.600000,141.100000,141.25000,1.783456e+06,2512.030000
75%,157.175000,159.400000,155.150000,156.925000,156.90000,2.813594e+06,4539.015000
max,327.700000,328.750000,321.650000,325.950000,325.75000,2.919102e+07,55755.080000


In [45]:
df.isna().sum()

Date                    0
Open                    0
High                    0
Low                     0
Last                    0
Close                   0
Total Trade Quantity    0
Turnover (Lacs)         0
dtype: int64

In [46]:
df.dtypes

Date                     object
Open                    float64
High                    float64
Low                     float64
Last                    float64
Close                   float64
Total Trade Quantity      int64
Turnover (Lacs)         float64
dtype: object

In [47]:
features = df.loc[:, ['Open', 'High', 'Low', 'Last', 'Total Trade Quantity', 'Turnover (Lacs)', 'Close']]
target = df.loc[:, 'Close']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)


In [48]:
def create_sequences(data, sequence_length=50):
    xs, ys = [], []
    for i in range(len(data) - sequence_length):
        x = data[i:i+sequence_length, :-1]  # All columns except 'Close' as features
        y = data[i+sequence_length, -1]  # 'Close' column as target
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)


In [49]:
sequence_length = 50
X, y = create_sequences(scaled_features, sequence_length)

# Shuffle and Split the data into training and testing sets (e.g., 70%-30% split)
indices = list(range(len(X)))
random.shuffle(indices)
split_point = int(0.70 * len(indices))

train_indices = indices[:split_point]
test_indices = indices[split_point:]

X_train, y_train = X[train_indices], y[train_indices]
X_test, y_test = X[test_indices], y[test_indices]


In [50]:
model_LSTM = Sequential()
model_LSTM.add(LSTM(units=50, return_sequences=True, input_shape=(sequence_length, X_train.shape[2])))
model_LSTM.add(LSTM(units=50))
model_LSTM.add(Dense(1))

# Compile the model
model_LSTM.compile(optimizer='adam', loss='mean_squared_error')

In [52]:
model_LSTM.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
44/44 [==============================] - 1s 19ms/step - loss: 2.6653e-04 - val_loss: 3.4640e-04
Epoch 2/50
44/44 [==============================] - 1s 20ms/step - loss: 2.8637e-04 - val_loss: 3.6853e-04
Epoch 3/50
44/44 [==============================] - 1s 18ms/step - loss: 2.4655e-04 - val_loss: 4.7241e-04
Epoch 4/50
44/44 [==============================] - 1s 18ms/step - loss: 2.6425e-04 - val_loss: 3.2442e-04
Epoch 5/50
44/44 [==============================] - 1s 18ms/step - loss: 2.3636e-04 - val_loss: 3.7178e-04
Epoch 6/50
44/44 [==============================] - 1s 18ms/step - loss: 3.0722e-04 - val_loss: 3.5122e-04
Epoch 7/50
44/44 [==============================] - 1s 18ms/step - loss: 2.5227e-04 - val_loss: 2.8421e-04
Epoch 8/50
44/44 [==============================] - 1s 18ms/step - loss: 2.3704e-04 - val_loss: 2.7575e-04
Epoch 9/50
44/44 [==============================] - 1s 18ms/step - loss: 2.2286e-04 - val_loss: 3.3158e-04
Epoch 10/50
44/44 [==================

In [57]:
predict_train = model_LSTM.predict(X_train)
MSE = mean_squared_error(predict_train, y_train)
R2 = r2_score(predict_train, y_train)
print('MSE', MSE)
print('R2', R2)

44/44 [==============================] - 0s 5ms/step
MSE 4.905271527120311e-05
R2 0.9986499458590568


In [58]:
predict_test = model_LSTM.predict(X_test)
MSE = mean_squared_error(predict_test, y_test)
R2 = r2_score(predict_test, y_test)
print('MSE', MSE)
print('R2', R2)

19/19 [==============================] - 0s 5ms/step
MSE 6.69533162408908e-05
R2 0.9983199266278754
